In [6]:
import mindspore, torch
import numpy as np

In [7]:
def judge(o1, o2, loss = 1e-3, prefix = '-'):
    """递归判断输出是否对齐"""
    prefix += '-'
    if (isinstance(o1, tuple)):
        for i in range(len(o1)):
            judge(o1[i], o2[i], loss=loss, prefix=prefix)
    elif (isinstance(o1,mindspore.Tensor)):
        print(f"{prefix}{np.allclose(o1.asnumpy(), o2.detach().numpy(), loss, loss)}")
    else:
        print(f"{type(o1)}-{type(o2)}:{o1==o2}")

In [8]:
from huggingface_hub import hf_hub_url

path = "/home/data/T5ckpt"

def download_script(size:str):
    """print wget to download files of a pretrained model"""
    print(f"wget {hf_hub_url(repo_id=size, filename='config.json')} -P {path}/{size}")
    print(f"wget {hf_hub_url(repo_id=size, filename='tokenizer.json')} -P {path}/{size}")
    print(f"wget {hf_hub_url(repo_id=size, filename='pytorch_model.bin')} -P {path}/{size}")

In [9]:
download_script("t5-small")

wget https://huggingface.co/t5-small/resolve/main/config.json -P /home/data/T5ckpt/t5-small
wget https://huggingface.co/t5-small/resolve/main/tokenizer.json -P /home/data/T5ckpt/t5-small
wget https://huggingface.co/t5-small/resolve/main/pytorch_model.bin -P /home/data/T5ckpt/t5-small


In [10]:
import logging
def torch_to_mindspore(pth_file, size:str=None):
    try:
        import torch
    except:
        raise ImportError(f"'import torch' failed, please install torch by "
                          f"`pip install torch` or instructions from 'https://pytorch.org'")

    size = "mindspore" if not size else size # rename ckpt

    from mindspore import Tensor
    from mindspore.train.serialization import save_checkpoint

    logging.info('Starting checkpoint conversion.')
    ms_ckpt = []
    state_dict = torch.load(pth_file, map_location=torch.device('cpu'))

    for k, v in state_dict.items():
        if 'shared.weight' in k:
            k = k.replace('shared.weight', 'decoder.embed_tokens.embedding_table')
        if 'relative_attention_bias.weight' in k:
            k = k.replace('relative_attention_bias.weight', 'relative_attention_bias.embedding_table')
        ms_ckpt.append({'name': k, 'data': Tensor(v.numpy())})

    ms_ckpt_path = pth_file.replace('.bin','.ckpt')
    ms_ckpt_path = ms_ckpt_path.replace('pytorch',size)
    try:
        save_checkpoint(ms_ckpt, ms_ckpt_path)
    except:
        raise RuntimeError(f'Save checkpoint to {ms_ckpt_path} failed, please checkout the path.')

    return ms_ckpt_path

In [11]:
torch_to_mindspore("/home/daiyuxin/ljm_script/codespace/t5_ckpt/t5-small/pytorch_model.bin", "t5-small")

'/home/daiyuxin/ljm_script/codespace/t5_ckpt/t5-small/t5-small_model.ckpt'